<a href="https://colab.research.google.com/github/tanviranjum21/DataScienceProjects/blob/master/Fifa_world_cup_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Description

The main goal of this project is to predict fifa world cup 2022 winners using Machine learning and by this prediction we will learn several concepts of Machine learning and Data Analysis. I will write every part why I am doing which part with detailed description so that It's easily understandable to the reader.

**Data Description**

For this analysis I am using two different dataset. One is "International football results from 1872 to 2022" and another one is "FIFA World Ranking 1992-2022".
Dataset 1 link: https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017

Dataset 2 link: https://www.kaggle.com/datasets/cashncarry/fifaworldranking

Using both of this dataset, we will predict the winner and also will do some cool visualization

In [2]:
!pip install featuretools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.1 MB/s 
     |████████████████████████████████| 207 kB 67.1 MB/s 
     |████████████████████████████████| 280 kB 55.9 MB/s 
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


# Importing Library

In [3]:
#Data Preprocessing and Exploration tool
import numpy as np
import pandas as pd
import re

#Data Visualization tool
import seaborn as sns
import matplotlib.pyplot as plt

#Feature Engineering
import featuretools as ft
from sklearn.feature_selection import SelectFromModel

#Machine learning
from sklearn import linear_model
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV


/usr/local/lib/python3.7/dist-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3.7 in next non-bugfix release.
  "Woodwork may not support Python 3.7 in next non-bugfix release.", FutureWarning
/usr/local/lib/python3.7/dist-packages/featuretools/__init__.py:67: FutureWarning: Featuretools may not support Python 3.7 in next non-bugfix release.
  FutureWarning,


# Data Loading and Preprocessing

In this part we will load the dataset and then preprocess and get ready the table as we need. As we will need to merge two different dataset into one single form

## **Loading Relevant Data**

In [4]:
world_cup_result =  pd.read_csv("/content/drive/MyDrive/Fifa_World_Cup_Analysis/Dataset/International_football_results/results.csv")
world_cup_goalscorers = pd.read_csv("/content/drive/MyDrive/Fifa_World_Cup_Analysis/Dataset/International_football_results/goalscorers.csv")
world_cup_shootouts = pd.read_csv("/content/drive/MyDrive/Fifa_World_Cup_Analysis/Dataset/International_football_results/shootouts.csv")
fifa_ranking = pd.read_csv("/content/drive/MyDrive/Fifa_World_Cup_Analysis/Dataset/fifa_ranking-2022-10-06.csv")

In [5]:
print(world_cup_result.info())
print(world_cup_goalscorers.info())
print(world_cup_shootouts.info())
print(fifa_ranking.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44152 entries, 0 to 44151
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        44152 non-null  object 
 1   home_team   44152 non-null  object 
 2   away_team   44152 non-null  object 
 3   home_score  44104 non-null  float64
 4   away_score  44104 non-null  float64
 5   tournament  44152 non-null  object 
 6   city        44152 non-null  object 
 7   country     44152 non-null  object 
 8   neutral     44152 non-null  bool   
dtypes: bool(1), float64(2), object(6)
memory usage: 2.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7667 entries, 0 to 7666
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       7667 non-null   object
 1   away_team  7667 non-null   object
 2   home_team  7667 non-null   object
 3   team       7667 non-null   object
 4   scorer     7667 non-null   obj

Here we can see the date fields in each datasets are in text (Object) format but it was supposed to be in date format. So we will change the date format to date from object. 

In [6]:
world_cup_result["date"] = pd.to_datetime(world_cup_result["date"])
world_cup_goalscorers["date"] = pd.to_datetime(world_cup_goalscorers["date"])
world_cup_shootouts["date"] = pd.to_datetime(world_cup_shootouts["date"])
fifa_ranking["rank_date"] = pd.to_datetime(fifa_ranking["rank_date"])

In [7]:
print(world_cup_result.info())
print(world_cup_goalscorers.info())
print(world_cup_shootouts.info())
print(fifa_ranking.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44152 entries, 0 to 44151
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        44152 non-null  datetime64[ns]
 1   home_team   44152 non-null  object        
 2   away_team   44152 non-null  object        
 3   home_score  44104 non-null  float64       
 4   away_score  44104 non-null  float64       
 5   tournament  44152 non-null  object        
 6   city        44152 non-null  object        
 7   country     44152 non-null  object        
 8   neutral     44152 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(2), object(5)
memory usage: 2.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7667 entries, 0 to 7666
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       7667 non-null   datetime64[ns]
 1   away_team  7667 non-null   

## **Cleaning N/A Values**

In [8]:
print("Number of missing value in World cup result dataset")
print(world_cup_result.isna().sum())
print("Number of missing value in World cup goalscorers dataset")
print(world_cup_goalscorers.isna().sum())
print("Number of missing value in World cup shootouts dataset")
print(world_cup_shootouts.isna().sum())
print("Number of missing value in fifa ranking dataset")
print(fifa_ranking.isna().sum())


Number of missing value in World cup result dataset
date           0
home_team      0
away_team      0
home_score    48
away_score    48
tournament     0
city           0
country        0
neutral        0
dtype: int64
Number of missing value in World cup goalscorers dataset
date         0
away_team    0
home_team    0
team         0
scorer       0
minute       0
own_goal     0
penalty      0
dtype: int64
Number of missing value in World cup shootouts dataset
date         0
home_team    0
away_team    0
winner       0
dtype: int64
Number of missing value in fifa ranking dataset
rank               0
country_full       0
country_abrv       0
total_points       0
previous_points    0
rank_change        0
confederation      0
rank_date          0
dtype: int64


From the analysis we can see that only in world cup result dataset has missing values in **home_score (48 values)** and **away_score (48 values)**. Otherwise the datasets has no missing values. To clean the dataset we can only drop the missing values as the number of missing is very low.

In [9]:
world_cup_result.dropna(inplace=True) #dropping NA values

We will take data from 2014 to present to understand and train the data according to that and we will explore a bit in their as well.

In [10]:
world_cup_result = world_cup_result.sort_values(by=['date'])
world_cup_goalscorers = world_cup_goalscorers.sort_values(by=['date'])
world_cup_shootouts = world_cup_shootouts.sort_values(by=['date'])
fifa_ranking = fifa_ranking.sort_values(by=['rank_date'])

In [11]:
world_cup_result = world_cup_result[(world_cup_result["date"] >= "2014-1-1")].reset_index(drop=True)
world_cup_goalscorers = world_cup_goalscorers[(world_cup_goalscorers["date"] >= "2014-1-1")].reset_index(drop=True)
world_cup_shootouts = world_cup_shootouts[(world_cup_shootouts["date"] >= "2014-1-1")].reset_index(drop=True)
fifa_ranking = fifa_ranking[(fifa_ranking["rank_date"] >= "2014-1-1")].reset_index(drop=True)

In [31]:
world_cup_result

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,2014-01-01,Kuwait,Jordan,1.0,2.0,WAFF Championship,Doha,Qatar,True
1,2014-01-04,Bahrain,Jordan,0.0,1.0,WAFF Championship,Doha,Qatar,True
2,2014-01-04,Namibia,Ghana,0.0,1.0,Friendly,Windhoek,Namibia,False
3,2014-01-04,Nigeria,Ethiopia,2.0,1.0,Friendly,Abuja,Nigeria,False
4,2014-01-04,Qatar,Kuwait,3.0,0.0,WAFF Championship,Doha,Qatar,False
...,...,...,...,...,...,...,...,...,...
7989,2022-11-16,Turkey,Scotland,2.0,1.0,Friendly,Diyarbakir,Turkey,False
7990,2022-11-16,Uzbekistan,Kazakhstan,2.0,0.0,Friendly,Tashkent,Uzbekistan,False
7991,2022-11-16,Moldova,Azerbaijan,1.0,2.0,Friendly,Chișinău,Moldova,False
7992,2022-11-16,Nepal,Pakistan,1.0,0.0,Friendly,Kathmandu,Nepal,False


In [12]:
conditions = [
    (world_cup_result['home_score'] > world_cup_result['away_score']),
    (world_cup_result['home_score'] < world_cup_result['away_score']),
    (world_cup_result['home_score'] == world_cup_result['away_score'])
]

values = [world_cup_result.home_team, world_cup_result.away_team,'draw']
world_cup_result['match_result_team'] = np.select(conditions, values)


In [13]:
world_cup_result

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,match_result_team
0,2014-01-01,Kuwait,Jordan,1.0,2.0,WAFF Championship,Doha,Qatar,True,Jordan
1,2014-01-04,Bahrain,Jordan,0.0,1.0,WAFF Championship,Doha,Qatar,True,Jordan
2,2014-01-04,Namibia,Ghana,0.0,1.0,Friendly,Windhoek,Namibia,False,Ghana
3,2014-01-04,Nigeria,Ethiopia,2.0,1.0,Friendly,Abuja,Nigeria,False,Nigeria
4,2014-01-04,Qatar,Kuwait,3.0,0.0,WAFF Championship,Doha,Qatar,False,Qatar
...,...,...,...,...,...,...,...,...,...,...
7989,2022-11-16,Andorra,Austria,0.0,1.0,Friendly,Malaga,Spain,True,Austria
7990,2022-11-16,Algeria,Mali,1.0,1.0,Friendly,Oran,Algeria,False,draw
7991,2022-11-16,Albania,Italy,1.0,3.0,Friendly,Tirana,Albania,False,Italy
7992,2022-11-16,Mexico,Sweden,1.0,2.0,Friendly,Girona,Spain,True,Sweden


Here in this particular analysis we have created a condition separately and then give values according to the logic and then map both of them using numpy. We could do the same task using filter functionality of pandas but dplyr of R has more exploratory friendly than pandas in my perspective in these cases

In [82]:
def highestPenaltyWinningTeam(from_date, to_date, dataset):
  filtered_data = dataset.loc[(dataset['date'] >= from_date) & (dataset['date'] < to_date)]
  filtered_data = filtered_data['winner'].value_counts().sort_values(ascending=False).reset_index(name = "value")
  return filtered_data

In [83]:
 Highest_Penalty_winning_Team = highestPenaltyWinningTeam('2020-01-01','2022-01-01',world_cup_shootouts)

In [89]:
 Highest_Penalty_winning_Team = Highest_Penalty_winning_Team.rename(columns = {'index':'Country_Name', 'value':'Number_of_win'})


In [90]:
Highest_Penalty_winning_Team

,Country_Name,Number_of_win
0,Italy,2
1,Scotland,2
2,Northern Ireland,1
3,Mexico,1
4,Honduras,1
5,Switzerland,1
6,Peru,1
7,Spain,1
8,Colombia,1
9,Slovakia,1
